## HTTP requests and APIs

Let's make a request to [JSONPlaceholder](https://jsonplaceholder.typicode.com/), a free fake API for testing:

In [ ]:
import requests

url = "https://jsonplaceholder.typicode.com/posts/1"
response = requests.get(url)

print(f"Status Code: {response.status_code}")

The response object contains everything about the server's reply. Here are the 4 most important attributes:

In [ ]:
print(f"Status Code: {response.status_code}")
print(f"JSON Response: {response.json()}")
print(f"Text Response: {response.text}")
print(f"Headers: {response.headers}")

In [ ]:
# Fetch multiple posts
response = requests.get("https://jsonplaceholder.typicode.com/posts")

if response.status_code == 200:
    posts = response.json()  # This is now a list of dictionaries
    print(f"Fetched {len(posts)} posts")

    first_post = posts[0]
    print(f"Title: {first_post['title']}")
else:
    print(f"Request failed with status: {response.status_code}")

## HTTP methods and parameters

### GET with query parameters

Query parameters let you filter, search, or customize your requests. They appear after the `?` in URLs.

In [ ]:
# Build the URL manually (not recommended)
url = "https://jsonplaceholder.typicode.com/posts?userId=1&_limit=5"
response = requests.get(url)

# Better: use the `params` argument
url = "https://jsonplaceholder.typicode.com/posts"
params = {
    'userId': 1,
    '_limit': 5
}

response = requests.get(url, params=params)
print(f"Final URL: {response.url}")  # See what was actually requested
print(f"Found {len(response.json())} posts")

In [ ]:
response = requests.get("https://jsonplaceholder.typicode.com/users/1")
user = response.json()

# Bracket notation access (will raise KeyError if a key is missing)
print(f"City: {user['address']['city']}")

# Using .get() with a default value - safe for deep nesting
lat = user.get('address', {}).get('geo', {}).get('lat', 'Unknown')
lng = user.get('address', {}).get('geo', {}).get('lng', 'Unknown')

print(f"Latitude: {lat}")  # None if any key is missing
print(f"Longitude: {lng}")

In [ ]:
response = requests.get("https://jsonplaceholder.typicode.com/posts")
posts = response.json()

# Extract specific fields from all posts
titles = [post['title'] for post in posts]
print(f"First 3 titles: {titles[:3]}")

# Filter by condition
user_1_posts = [post for post in posts if post['userId'] == 1]
print(f"User 1 has {len(user_1_posts)} posts")

# Extract nested data from a list
response = requests.get("https://jsonplaceholder.typicode.com/users")
users = response.json()

cities = [user['address']['city'] for user in users]
print(f"Cities: {cities}")

In [ ]:
# with statement allows to set headers once, use for all requests
with requests.Session() as session:
    session.headers.update({
        'User-Agent': 'My App/1.0',
        'Accept': 'application/json'
    })

    # All requests in this session use these headers
    response1 = session.get("https://jsonplaceholder.typicode.com/posts/1")
    response2 = session.get("https://jsonplaceholder.typicode.com/users/1")

    print(f"Post: {response1.json()['title']}")
    print(f"User: {response2.json()['name']}")

Context managers (`with` statement) automatically handle resource cleanup. When you use with `requests.Session()`, Python creates a session object, lets you use it for multiple requests, and automatically closes it when done — even if an error occurs.

Sessions reuse TCP connections between requests instead of creating new ones each time, which is faster. You can also set headers, cookies, or authentication once and they'll apply to all requests. When the `with` block ends, the session closes automatically.

### POST - Creating resources

POST requests send data to the server to create new resources. Use the `json` parameter to automatically serialize Python dictionaries.

In [ ]:
url = "https://jsonplaceholder.typicode.com/posts"

new_post = {
    'title': 'My New Post',
    'body': 'This is the content of my post',
    'userId': 1
}

try:
    response = requests.post(url, json=new_post, timeout=5)
    response.raise_for_status()  # Raises HTTPError for bad status codes

    created_post = response.json()
    print(f"Status: {response.status_code}")  # 201 = Created
    print(f"Created post with ID: {created_post['id']}")

except requests.exceptions.HTTPError as e:
    print(f"HTTP Error: {e}")

**What happens here:**
- The `json` parameter automatically converts your dict to JSON
- Sets the `Content-Type: application/json` header

The `try` block contains code that might fail. If an error occurs, Python jumps to the matching except block instead of crashing. This lets you handle errors gracefully and provide feedback to users.
You can catch specific exceptions (like `HTTPError` above) or use multiple except blocks to handle different error types differently. Try to avoid using bare
```python
except Exception as e
```
it catches everything and tends to be harder to debug.


In [ ]:
# For form-encoded data (like HTML forms)
form_data = {
    'username': 'john',
    'email': 'john@example.com'
}

response = requests.post(url, data=form_data)  # Use 'data' instead of 'json'

### PUT - Full updates (replacing entire resource)

PUT replaces the entire resource with your new data.

In [ ]:
url = "https://jsonplaceholder.typicode.com/posts/1"

updated_post = {
    'id': 1,
    'title': 'Updated Title',
    'body': 'Completely new content',
    'userId': 1
}

response = requests.put(url, json=updated_post)

print(f"Status: {response.status_code}")
print(response.json())

### PATCH - Partial updates (changing specific fields)

PATCH updates only the fields you specify, leaving others unchanged.

In [ ]:
url = "https://jsonplaceholder.typicode.com/posts/1"

partial_update = {
    'title': 'Just changing the title'
}

response = requests.patch(url, json=partial_update)

print(f"Status: {response.status_code}")
updated = response.json()
print(f"New title: {updated['title']}")
print(f"Body unchanged: {updated['body']}")  # Original body remains

**PUT vs PATCH:**
- **PUT**: "Replace everything with this" (send all fields)
- **PATCH**: "Just update these specific fields" (send only what changes)

### DELETE - Removing resources

In [ ]:
url = "https://jsonplaceholder.typicode.com/posts/1"

response = requests.delete(url)

print(f"Status: {response.status_code}")  # 200 or 204 (No Content)
print("Post deleted successfully" if response.status_code in [200, 204] else "Failed")

### The basic pattern: `requests.METHOD(url, **kwargs)`

In [ ]:
# Common keyword arguments (kwargs) across all methods:
response = requests.get(url, params={'key': 'value'})      # Query parameters
response = requests.post(url, json={'key': 'value'})       # JSON body
response = requests.put(url, json={'key': 'value'})        # JSON body
response = requests.patch(url, json={'key': 'value'})      # JSON body
response = requests.delete(url)                             # Usually no body

# Other useful keyword arguments (work with any method):
response = requests.get(
    url,
    headers={'Authorization': 'Bearer token123'},  # Custom headers
    timeout=5,                                      # Timeout in seconds
    params={'filter': 'active'}                     # Query params
)

##  Headers and Authentication

Before discussing authentication, we have to mention managing sensitive credentials. This is done using `.env` files with the `python-dotenv` library. Never hardcode API keys or tokens directly in your code. Don't forget to add `.env` files to `.gitignore` (this is already present in this repository [since it uses the official Python `.gitignore`](https://github.com/github/gitignore/blob/fc6ce5da28a8c3480cc8a5acad050449f72a9261/Python.gitignore#L150), but if you create `.env.dev`, `.env.prod`, or something similar, don't forget to wildcard them or add them to the `.gitignore` explicitly).

Let's use the GitHub API to fetch public information about your starred repositories and your user profile. This doesn't modify anything.

Getting a GitHub token:

1. Go to GitHub Settings → Developer settings → Personal access tokens → Tokens (classic)    
2. Click "Generate new token (classic)"    
3. Give it a name like "Requests tutorial"      
4. Select no scopes or just public_repo (read-only access to public repos)
5. Copy the `.env.template` to a `.env` file:
```shell
cp .env.template .env
```     
6. Generate and copy the token to the created `.env` file      

In [ ]:
import os
from dotenv import load_dotenv
import requests

load_dotenv('/workspace/.env', override=True)
github_token = os.getenv('GITHUB_TOKEN')

In [ ]:
# GitHub API endpoint
url = "https://api.github.com/user"

headers = {
    'Authorization': f'Bearer {github_token}',     # Authentication
    'Accept': 'application/vnd.github.v3+json',   # Expected response format
    'Content-Type': 'application/json'            # Format of data you're sending
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    user = response.json()
    print(f"Username: {user['login']}")
    print(f"Name: {user['name']}")
    print(f"Public repos: {user['public_repos']}")
    print(f"Followers: {user['followers']}")
else:
    print(f"Error: {response.status_code}")

You can see exactly what headers were sent with your request:

In [ ]:
# View the headers you actually sent
print("Headers sent with request:")
print("=" * 60)
for key, value in response.request.headers.items():
    # Mask sensitive data
    if key == 'Authorization':
        print(f"{key}: Bearer ***hidden***")
    else:
        print(f"{key}: {value}")

The server sends back headers:

In [ ]:
print("Response headers:")
print("=" * 60)

# Access specific headers
print(f"Content-Type: {response.headers.get('Content-Type')}")
print(f"Server: {response.headers.get('Server', 'Unknown')}")
print(f"Status: {response.status_code}")

# GitHub-specific rate limit headers
print("\nGitHub Rate Limit Info:")
print(f"Limit: {response.headers.get('X-RateLimit-Limit')}")
print(f"Remaining: {response.headers.get('X-RateLimit-Remaining')}")
print(f"Reset: {response.headers.get('X-RateLimit-Reset')}")

# View all response headers
print("\nAll response headers:")
for key, value in response.headers.items():
    print(f"{key}: {value}")

HTTP headers are case-insensitive, but `requests` handles this automatically.

## Introduction to async Python

### Why async?

When you make HTTP requests synchronously, your program sits idle waiting for responses. This is called "blocking I/O." With async, you can start multiple requests and let them run concurrently. While waiting for one response, you can start other requests.

```
Request 1 → Wait ──┐
Request 2 → Wait ──┤
Request 3 → Wait ──┼→ All responses come back around the same time
Request 4 → Wait ──┤
Request 5 → Wait ──┘
```

### Async basics

 `async def` defines an async function:

In [ ]:
# Regular function
def fetch_data():
    return requests.get(url)

# Async function
async def fetch_data():
    return await session.get(url)

Functions defined with `async def` return coroutines that must be awaited. `await` waits for async operations to complete:

In [ ]:
async def get_user(session, username):
    response = await session.get(f"https://api.github.com/users/{username}")
    data = await response.json()  # await here also
    return data

Use `await` before any async operation (network calls, file I/O, etc.). Without `await`, you get a coroutine object instead of the actual result:

In [ ]:
import aiohttp
import asyncio
import time

async def get_username(session):
    async with session.get("https://api.github.com/users/torvalds") as response:
        data = response.json()  # <- missing await
        return data

async def main():
    async with aiohttp.ClientSession() as session:
        result = await get_username(session)
        print(result)  # Prints: <coroutine object ClientResponse.json at 0x...>

await main()

If you see `<coroutine object <name> at 0x...>`, the `await` is missing:

In [ ]:
async def get_username(session):
    async with session.get("https://api.github.com/users/torvalds") as response:
        data = await response.json() # <- added the await
        return data

async def main():
    async with aiohttp.ClientSession() as session:
        result = await get_username(session)
        print(result)  # Prints: <coroutine object ClientResponse.json at 0x...>

await main()

Jupyter runs an event loop in the background automatically, so we use `await` to call async code from async code. Use `asyncio.run()` to start async code from sync code (regular Python scripts):

```python
# If you execute this in a code cell, you'll get the following:
# RuntimeError: asyncio.run() cannot be called from a running event loop
async def main():
    async with aiohttp.ClientSession() as session:
        user = await get_user(session, "torvalds")
        print(user['name'])

asyncio.run(main())
```

`asyncio.run()` vs `await`

| Feature | `asyncio.run()` | `await` |
|---------|----------------|---------|
| **Event loop** | Creates and closes a new event loop | Uses an existing event loop |
| **Where to use** | Regular Python scripts (no loop exists) | Inside async functions |
| **Usage frequency** | Once per script (from sync code) | Multiple times (from async code) |
| **Resource cleanup** | Cleans up resources when done | Doesn't create or close the loop |

The code below fetches 5 GitHub user profiles at the same time, showing how async works in practice.    
Note on `asyncio.gather()`: This runs multiple tasks concurrently and waits for all of them to finish. Results come back in the same order you passed them in.

In [ ]:
urls = [
    "https://api.github.com/users/torvalds",
    "https://api.github.com/users/gvanrossum",
    "https://api.github.com/users/kennethreitz",
    "https://api.github.com/users/django",
    "https://api.github.com/users/pallets"
]

async def fetch_user(session, url):
    """Fetch a single user profile"""
    async with session.get(url) as response:
        user = await response.json()
        return user

async def fetch_all_users():
    """Fetch all users concurrently"""
    print("Fetching user profiles asynchronously...\n")
    start = time.time()

    async with aiohttp.ClientSession() as session: #  requests.Session() persists auth across requests
        # Create tasks for all requests
        tasks = [fetch_user(session, url) for url in urls]

        # Run all tasks concurrently
        users = await asyncio.gather(*tasks)

        for user in users:
            print(f"✓ Fetched {user['login']} - {user['public_repos']} repos")

    elapsed = time.time() - start
    print(f"\nTotal time: {elapsed:.2f} seconds")

await fetch_all_users()

### Rate limiting

Most APIs limit how many requests you can make per hour. GitHub allows 60 requests/hour without authentication, or 5,000/hour with a token. Since async code fires off requests simultaneously, you can exhaust your quota in seconds. To avoid this, monitor rate limit headers and either space out your requests or handle rate limit errors when they happen:

In [ ]:
import aiohttp
import asyncio
from datetime import datetime

async def fetch_repos_safely(session, username, headers):
    """Fetch repos with full rate limit handling"""
    url = f"https://api.github.com/users/{username}/repos"

    async with session.get(url, headers=headers) as response:
        # Check rate limits
        remaining = int(response.headers.get('X-RateLimit-Remaining', 0))
        limit = response.headers.get('X-RateLimit-Limit')
        reset_timestamp = int(response.headers.get('X-RateLimit-Reset', 0))

        print(f"Fetching {username}'s repos | Rate limit: {remaining}/{limit}")

        # Handle rate limit
        if response.status == 429:
            reset_time = datetime.fromtimestamp(reset_timestamp)
            wait_seconds = (reset_time - datetime.now()).total_seconds()
            print(f"Rate limited! Waiting {wait_seconds:.0f} seconds...")
            await asyncio.sleep(wait_seconds)
            return await fetch_repos_safely(session, username, headers)

        # Warn if running low
        if remaining < 10:
            reset_time = datetime.fromtimestamp(reset_timestamp)
            print(f"Only {remaining} requests left! Resets at {reset_time}")

        response.raise_for_status()
        return await response.json()

async def main():
    headers = {
        'Authorization': f'Bearer {github_token}',
        'Accept': 'application/vnd.github.v3+json'
    }

    usernames = ["torvalds", "gvanrossum", "kennethreitz"]

    async with aiohttp.ClientSession() as session:
        # Create all tasks at once
        tasks = [fetch_repos_safely(session, username, headers) for username in usernames]

        # Run them concurrently
        results = await asyncio.gather(*tasks)

        for username, repos in zip(usernames, results):
            print(f"  → {username} has {len(repos)} public repos\n")

await main()